<a href="https://colab.research.google.com/github/MihaiDogariu/Vodafone-Summer-School/blob/main/DL_Crash_Course_Apps_1_%26_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clasificarea imaginilor
Acest proiect implementeaza arhitectura AlexNet pentru clasificarea imaginilor reale și image retrieval. Baza de date aleasa pentru demonstratie este CIFAR10.

## #TODO:
1. Completați arhitectura rețelei cu modelul AlexNet
1. Scrieți funcția de calcul a distanței între 2 tensori
1. Rulați inferența(clasificarea) pe o imagine captată cu ajutorul camerei
1. Găsiți cea mai asemănătoare imagine cu imaginea captată cu ajutorul camerei
laptop-ului.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

# Alegem configuratia sistemului (cpu/gpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setam media si deviatia standard pentru normalizarea bazei de date - acestea sunt calculate la nivel de canal si doar pe baza de date de antrenare!
normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])

##1. Pre-procesarea datelor

In [ ]:
def get_train_valid_loader(data_dir,
                           batch_size,
                           augment,
                           random_seed,
                           normalize,
                           valid_size=0.1,
                           shuffle=True):

    # Definim setul de transformari necesare bazei de date
    valid_transform = transforms.Compose([
            transforms.Resize((227,227)), # baza de date CIFAR10 contine imagini de dimensiunea 32x32, iar AlexNet are intrari de dimensiune 227x227
            transforms.ToTensor(),        # transformarea intrarilor in tensori
            normalize,                    # aplicarea normalizarii
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4), # decuparea unor regiuni aleatoare de dimensiune 32x32 din imaginea originala la care s-a adaugat padding=4
            transforms.RandomHorizontalFlip(0.4), # oglindirea imaginilor cu probabilitate de 40%
            transforms.Resize((227,227)),         # redimensionarea imaginilor augmentate la dimensiunea de 227x227 pixeli
            transforms.ToTensor(),                # transformarea intrarilor in tensori
            normalize,                            # aplicarea normalizarii
        ])
    else:
        train_transform = valid_transform

    # Fiind o baza de date foarte populara, CIFAR10 poate fi descarcata cu ajutorul modulului torchvision
    train_dataset = datasets.CIFAR10(root=data_dir,
                                     train=True,
                                     download=True,
                                     transform=train_transform,
                                     )

    valid_dataset = datasets.CIFAR10(root=data_dir,
                                     train=True,
                                     download=True,
                                     transform=valid_transform,
                                     )

    # Alegem numarul de esantioane pentru train/val
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    # Amestecam indecsii
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    # Separam indecsii de train in train+val
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # Cream dataloaders pentru train si val
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)


def get_test_loader(data_dir,
                    batch_size,
                    normalize,
                    shuffle=True):

    # Transformari asemanatoare cu cele pentru train/val. Normalizarea se face cu aceleasi valori ca in cazul train!
    transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        normalize,
    ])

    # Descarcarea bazei de test
    dataset = datasets.CIFAR10(
        root=data_dir, train=False,
        download=True, transform=transform,
    )

    # Crearea dataloader pentru test
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle
    )

    return data_loader

def get_dataset(data_dir):

    # Transformari asemanatoare cu cele pentru train/val. Normalizarea se face cu aceleasi valori ca in cazul train!
    transform = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
    ])

    # Descarcarea bazei de test
    dataset = datasets.CIFAR10(
        root=data_dir, train=False,
        download=True, transform=transform,
    )

    return dataset


# Crearea efectiva a dataloaders
train_loader, valid_loader = get_train_valid_loader(
    data_dir = './data',
    batch_size = 64,
    augment = True,
    random_seed = 1,
    normalize = normalize
)

test_loader = get_test_loader(
    data_dir = './data',
    batch_size = 64,
    normalize = normalize
)

dataset = get_dataset(
     data_dir = './data'
)

infer_transform = transforms.Compose([
                  transforms.Resize((227,227)),
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
                  ])

label_to_strings = {0:"airplane",
                    1:"automobile",
                    2:"bird",
                    3:"cat",
                    4:"deer",
                    5:"dog",
                    6:"frog",
                    7:"horse",
                    8:"ship",
                    9:"truck"}

##2. Definirea modelului

Resurse utile (documentatii):
- strat convolutional: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
- strat complet conectat: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html?highlight=linear#torch.nn.Linear
- strat max pooling: https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html?highlight=maxpool#torch.nn.MaxPool2d
- activare ReLU: https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html?highlight=relu#torch.nn.ReLU
- regularizare dropout: https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html?highlight=dropout#torch.nn.Dropout
- mod secvential de compunere a straturilor: https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html?highlight=sequential#torch.nn.Sequential

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))


        # TODO: de completat cu restul straturilor retelei


        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def embedding(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        return out

    def forward(self, x):
        out = self.embedding(x)
        out = self.fc2(out)
        return out

##3. Antrenarea rețelei

In [ ]:
# Alegerea hiperparametrilor
num_classes = 10
num_epochs = 2
batch_size = 64
learning_rate = 0.005

# Trecerea modelului pe gpu
model = AlexNet(num_classes).to(device)

# Alegerea functiei de pierdere. Clasificare de imagini => cross-entropy
criterion = nn.CrossEntropyLoss()
# Alegerea optimizatorului
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

In [ ]:
# Antrenarea modelului
total_step = len(train_loader)

for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):
        # Incarcam tensorii pe gpu/cpu
        images = images.to(device)
        labels = labels.to(device)

        # Forward propagation
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop si rularea unui pas de optimizare a ponderilor
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Rularea algoritmului pe baza de validare
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))


##4. Testarea rețelei

In [ ]:
# Rularea algoritmului pe baza de test
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

## Crearea bazei de vectori de trăsături pentru Image Retrieval

In [ ]:
# Crearea unei baze de date cu vectori de trasaturi
feats = {}
model.eval()
with torch.no_grad():
    for i in range(len(dataset)):
        image, label = dataset[i]
        image = image[None, :].to(device)
        feats[i] = model.embedding(image)

#Funcții Ajutătoare

In [ ]:
def infer_image(image):
  with torch.no_grad():
    image = infer_transform(image)
    image = image[None, :].to(device)
    model.eval()
    output = model(image)
    print (output)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    predicted_class = torch.argmax(probabilities)
    return probabilities, predicted_class

def distance(tensor1, tensor2):
  # TODO: de implementat distanta Euclidiana intre 2 tensori
  pass

def find_best_match(image):
  with torch.no_grad():
    image = infer_transform(image)
    image = image[None, :].to(device)
    model.eval()
    query_feat = model.embedding(image)
    print(query_feat)
    min_dist = 999999
    best_idx = -1
    for (idx, feature) in feats.items():
      torch_dist = distance(query_feat, feature)
      if torch_dist < min_dist:
        min_dist = torch_dist
        best_idx = idx
        print(best_idx, min_dist)
    return best_idx

def show_image_at_idx(idx):
  image, label = dataset[idx]
  image = np.transpose(image, (1, 2, 0))
  plt.imshow(image)
  plt.title(label_to_strings[label])
  plt.show()

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  # display(Image(filename))
  im = Image.open("photo.jpg")
  plt.imshow(im)
  plt.show()
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))